In [ ]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import shutil
import os

In [ ]:
train = pd.read_csv('../input/plant-pathology-2021-fgvc8/train.csv')
print(train.shape)
train.head()

In [ ]:
seed = 32
target_size = (380, 380)
batch_size = 16
test_img = '../input/plant-pathology-2021-fgvc8/test_images'
submission = pd.read_csv('../input/plant-pathology-2021-fgvc8/sample_submission.csv')
submission.head()

In [ ]:
import cv2
import numpy as np
def get_cut_image(image):
    img = cv2.blur(image,(3,3))
    copy = np.uint8(img)
    canny = cv2.Canny(copy, 145, 165)
    box = np.argwhere(canny>0)
    y1,x1 = box.min(axis=0)
    y2,x2 = box.max(axis=0)
    cut_img = img[y1:y2, x1:x2]
    cut_img = cv2.resize(cut_img, target_size)
    cut_img = cut_img.astype("float32")*(1.)/255
    return np.array(cut_img)

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

pre = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[0.5, 1.5],
    rotation_range=45,
    shear_range=0.2,
    zoom_range=0.2, 
    horizontal_flip=True,
    vertical_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2)
# pos = ImageDataGenerator(
# #     rescale=1./255,
#     brightness_range=[0.5, 1.5],
#     rotation_range=45,
#     shear_range=0.2,
#     zoom_range=0.2, 
#     horizontal_flip=True,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     preprocessing_function = get_cut_image)

In [ ]:
# # # TTA
pos = ImageDataGenerator(
#     rescale=1./255,
    brightness_range=[0.5, 1.5],
    rotation_range=15, #45
    shear_range=0.2,
    zoom_range=0.3, #0.2 
    featurewise_center=False, #
    featurewise_std_normalization=False, #
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    vertical_flip=True,
    validation_split= 0.2,
    preprocessing_function = get_cut_image)

In [ ]:
test_generator = pos.flow_from_dataframe(
                  submission,
                  directory = test_img,
                  x_col = 'image',
                  y_col = 'labels',
                  class_mode = "raw",
                  batch_size=batch_size,
                  target_size = target_size,
                  color_mode="rgb",
                  shuffle = False,
                  seed = seed,)

In [ ]:
import keras
def load_model():
    model_pre = keras.models.load_model("../input/k3-0726/B4(oversample)4Fold_cv_BC_2553.h5",compile=False) #compile=False
    return model_pre

In [ ]:
model = load_model()
pred = model.predict(test_generator)

## TTA

In [ ]:
model = load_model()
tta_steps = 5
predictions = []

for i in range(tta_steps):
    preds = model.predict(test_generator)
    predictions.append(preds)

pred = np.mean(predictions, axis=0)

In [ ]:
perdict = (pred>0.33)
n_label = ['complex', 'frog_eye_leaf_spot', 'healthy', 'powdery_mildew', 'rust', 'scab']
answer = []

for i in range(perdict.shape[0]):
    temp = []
    for j, k in enumerate(n_label):
        if perdict[i, j]:
            temp.append(k)
    answer.append(temp)
    
answer = [' '.join(n) for n in answer]

In [ ]:
np.around(pred, decimals=3, out=None)

In [ ]:
submission['labels'] = np.array(answer)
submission

In [ ]:
submission.to_csv('submission.csv', index=False)